In [9]:
import cv2
import glob
import numpy as np
import numpy
import os
import matplotlib.pyplot as plt

from win32gui import GetWindowText, GetForegroundWindow, GetWindowRect, FindWindow
import pyscreenshot as ImageGrab
import time
import os
from IPython.display import display, clear_output
import datetime
from KartriderKit.Utils.location import locateOnPicture, locateCenterOnPicture
'''
# if the foreground window is as the same as desired_window, then this script will capture your current screen and check current scenario.
# which is faster than capture screenshot using Nox.
'''
WORKPATH = "D:/GameProjects/Kartrider/SmartKartRider"
desired_window = "BlueStacks App Player"
adb_port = 52883
os.chdir(WORKPATH)

In [10]:
target_size = (1280,720)
model_input_shape = (299,299)
first_classes = ['dialog', 'loading', 'others', 'reward']
second_classes = ['idle', 'inQuest']

In [11]:
# The Nox install path

'''
The simulator default port, if you using multiple simulator, you should change the port.
You can check your port number use : os.popen('adb devices').read()
'''
print(os.popen('adb devices').read())
connect = os.popen("adb connect 127.0.0.1:" + str(adb_port)).read()
print(connect)

List of devices attached


connected to 127.0.0.1:52883



In [25]:
def read_and_resize(path,target_size):
    return cv2.resize(cv2.imread(path),target_size)/255

def crop_img(inputImg, scenario,model_input_shape):
    if scenario == 'reward':
        return inputImg[:,335:940]
    elif scenario == 'dialog':
        return inputImg[460:,]
    elif scenario == 'questArrow':
        return inputImg[460:,]
    elif scenario == 'autoIcon':
        icon = inputImg[632:698,351:416]
        icon = cv2.resize(icon,model_input_shape)
        return icon

In [84]:
os.system('adb shell screencap -p /sdcard/screenshot.png')
os.system('adb pull /sdcard/screenshot.png ./screenshot.png')
im = cv2.imread('./screenshot.png')
im

array([[[ 17,  12,   9],
        [ 23,  16,  15],
        [ 17,  16,   9],
        ...,
        [149,  54,  35],
        [149,  53,  33],
        [152,  54,  35]],

       [[ 23,  19,  16],
        [ 17,  16,  16],
        [ 24,  19,  16],
        ...,
        [147,  53,  35],
        [150,  55,  35],
        [147,  53,  35]],

       [[ 17,  16,   9],
        [ 23,  19,  15],
        [ 17,  13,   9],
        ...,
        [146,  53,  35],
        [146,  52,  32],
        [145,  53,  35]],

       ...,

       [[ 24,  16,  15],
        [ 24,  13,   9],
        [ 24,  19,  15],
        ...,
        [  9,   4,   8],
        [ 15,  11,   8],
        [  9,   8,   8]],

       [[ 24,  12,   9],
        [ 24,  16,  15],
        [ 24,  12,   9],
        ...,
        [  8,   7,   7],
        [  8,   4,   1],
        [  8,   7,   7]],

       [[ 24,  19,  15],
        [ 24,  16,   9],
        [ 24,  16,  15],
        ...,
        [  9,   8,   8],
        [ 15,   8,   8],
        [  9,   4,   8]]

In [52]:
im = cv2.imread('./screenshot.png')
# im = cv2.imread('./raw_data/Screenshot_2022.10.22_11.22.30.011.png')
subIm = cv2.imread("D:/GameProjects/Kartrider/SmartKartRider/raw_data/US/Button/Close-Window.PNG")
retVal = locateCenterOnPicture(subIm, im)
retVal

Point(x=1254, y=24)

In [49]:
str(retVal[0]) + " " + str(retVal[1])

'1254 24'

In [50]:
locateOnPicture(subIm, im)

Box(left=1236, top=6, width=37, height=37)

In [53]:
if locateOnPicture(subIm, im):
    print("Close Window")
    retVal = locateCenterOnPicture(subIm, im)
    os.system("adb shell input tap " + str(retVal[0]) + " " + str(retVal[1]))
    time.sleep(2)

Close Window


In [ ]:
# run_count = 0
# while True:
#     clear_output(wait=True)
#     run_count += 1
#     print(f"Current round count:{run_count}")
#     if run_count%500==0:
#         #restart adb server and connection after a while
#         os.system("adb shell kill-server")
#         os.system("adb shell start-server")
#         os.popen("adb connect 127.0.0.1:" + str(adb_port))
#     print(datetime.datetime.now())
#     s = time.time()
#     current_window = GetWindowText(GetForegroundWindow())
#
#     # If current_window is not as desired_window `夜神模擬器`, capture screen use adb tools
#     if current_window != desired_window:
#         try:
#             print("ForegroundWindow not BlueStacks, adb screencap is using.")
#             os.system('adb shell screencap -p /sdcard/screenshot.png')
#             os.system('adb pull /sdcard/screenshot.png ./screenshot.png')
#             im = cv2.imread('./screenshot.png')
#             os.system('adb shell rm /sdcard/screenshot.png')
#             os.remove('./screenshot.png')
#         except:
#             print("Restart adb service")
#             os.system("adb shell kill-server")
#             os.system("adb shell start-server")
#             os.popen("adb connect 127.0.0.1:" + str(adb_port))
#             continue
#     else:
#         rect = GetWindowRect(FindWindow(None, desired_window))
#         im = ImageGrab.grab(bbox=(rect[0],rect[1]+35,rect[2],rect[3]))
#         im = np.array(im)
#     if locateOnPicture('accept_button.png'):
#         pyautogui.click(pyautogui.locateCenterOnScreen('accept_button.png'))
#
#     elif locateOnPicture('confirm_button.png'):
#         pyautogui.click(pyautogui.locateCenterOnScreen('confirm_button.png'))
#
#     elif locateOnPicture('complete_button.png'):
#         click(pyautogui.locateCenterOnScreen('complete_button.png'))
#
#     elif locateOnScreen('claim_reward.png'):
#         pyautogui.click(pyautogui.locateCenterOnScreen('claim_reward.png'))
#
# ###################################################################################################
#     # "SKIP" AND "START-QUEST" buttons are below
#     # click "Start Quest" button
#     elif pyautogui.pixelMatchesColor(579, 892, (210, 195, 140)):
#         pyautogui.click(284, 402)
#         time.sleep(1)
#
#     # click "Skip" Button
#     elif pyautogui.pixelMatchesColor(308, 128, (246, 240, 239)):
#         pyautogui.click(308, 128)
#         time.sleep(0.5)
#
#     else:
#         time.sleep(5)


# Test

## begin game

In [13]:
# im = cv2.imread('./raw_data/US/TestPictures/StartGame-Scenario.png')
im = cv2.imread('./raw_data/US/TestPictures/StartGame-Scenario.png')
subIm = cv2.imread("./raw_data/US/Button/StartGame-Scenario-Start.png")

if locateOnPicture(subIm, im):
    print("Begin Game")
    retVal = locateCenterOnPicture(subIm, im)
    os.system("adb shell input tap " + str(retVal[0]) + " " + str(retVal[1]))
    # os.system("adb shell input tap 635 635")
    time.sleep(1)

Begin Game
Begin Game
Begin Game
Begin Game
Begin Game
Begin Game
Begin Game



KeyboardInterrupt



## Close Main window

In [11]:
im = cv2.imread('./raw_data/US/TestPictures/MainWindows-1.png')
subIm = cv2.imread("./raw_data/US/Button/Close-Window.PNG")

if locateOnPicture(subIm, im):
    print("Close Window")
    retVal = locateCenterOnPicture(subIm, im)
    os.system("adb shell input tap " + str(retVal[0]) + " " + str(retVal[1]))
    time.sleep(2)

Close Window


## Close Sub Window

In [12]:
im = cv2.imread('./raw_data/US/TestPictures/SubWindows-1.png')
subIm = cv2.imread("./raw_data/US/Button/Close-Event-Center.png")

if locateOnPicture(subIm, im):
    print("Close Window")
    retVal = locateCenterOnPicture(subIm, im)
    os.system("adb shell input tap " + str(retVal[0]) + " " + str(retVal[1]))
    time.sleep(2)

Close Window


## Start Race

In [13]:
im = cv2.imread('./raw_data/US/TestPictures/Main-Scenario.png')
subIm = cv2.imread("./raw_data/US/Button/Main-Scenario-StartGame.png")

if locateOnPicture(subIm, im):
    print("Close Window")
    retVal = locateCenterOnPicture(subIm, im)
    os.system("adb shell input tap " + str(retVal[0]) + " " + str(retVal[1]))
    time.sleep(2)

Close Window


## Select Time Trial

In [18]:
im = cv2.imread('./raw_data/US/TestPictures/GameSelection-Scenario.png')
subIm = cv2.imread("./raw_data/US/Button/GameSelection-Scenario-TimeTrial.png")

if locateOnPicture(subIm, im):
    print("Close Window")
    retVal = locateCenterOnPicture(subIm, im)
    os.system("adb shell input tap " + str(retVal[0]) + " " + str(retVal[1]))
    time.sleep(2)

Close Window


## Begin A time Trial

In [24]:
im = cv2.imread('./raw_data/US/TestPictures/TimeTrial-Scenario.png')
subIm = cv2.imread("./raw_data/US/Button/TimeTrial-Scenario-Start.png")

if locateOnPicture(subIm, im, confidence = 0.5):
    print("Close Window")
    retVal = locateCenterOnPicture(subIm, im, confidence = 0.5)
    os.system("adb shell input tap " + str(retVal[0]) + " " + str(retVal[1]))
    time.sleep(2)

Close Window


## Back To TimeTrial Scenario

In [ ]:
# im = cv2.imread('./raw_data/US/TestPictures/TimeTrial-Scenario.png')
# subIm = cv2.imread("./raw_data/US/Button/TimeTrial-Scenario-Start.png")
#
# if locateOnPicture(subIm, im, confidence = 0.5):
#     print("Close Window")
#     retVal = locateCenterOnPicture(subIm, im, confidence = 0.5)
#     os.system("adb shell input tap " + str(retVal[0]) + " " + str(retVal[1]))
#     time.sleep(2)